# Stochastic LD Correction Benchmark

## Does the SER variance inflation correction control FDR when using stochastic LD?

When individual-level genotype data is unavailable, SuSiE-RSS can use a **stochastic LD sketch** to approximate the LD matrix. The sketch is formed by random projection:

$$\hat{R} = \frac{1}{B} U U^\top, \quad U = X_{\text{std}}^\top W, \quad W \sim N(0, I_B / n)$$

where $X_{\text{std}}$ is the column-standardized genotype matrix ($n \times p$) and $B$ is the sketch size.

However, the noise in $\hat{R}$ inflates the variance of the Bayes factor under the null, leading to **elevated false discovery rate (FDR)** if uncorrected. The `stochastic_ld_sample` parameter in `susie_rss()` activates a variance inflation correction in the single-effect regression (SER) that accounts for this noise.

This notebook benchmarks the correction across different sketch sizes $B \in \{2000, 5000, 10000\}$, comparing:
1. **Gold standard** — in-sample LD ($R = \text{cor}(X)$)
2. **Subsample** — $R$ from a subsample of $B$ individuals
3. **Stochastic sketch, no correction** — sketch $U$ passed as `X`, no `stochastic_ld_sample`
4. **Stochastic sketch, with correction** — sketch $U$ passed as `X`, `stochastic_ld_sample = B`
5. **Stochastic sketch + NIG prior, no correction** — sketch $U$ with `estimate_residual_method = "Servin_Stephens"`, no `stochastic_ld_sample`
6. **Stochastic sketch + NIG prior, with correction** — sketch $U$ with `estimate_residual_method = "Servin_Stephens"` and `stochastic_ld_sample = B`

Methods 5–6 use the Normal-Inverse-Gamma (NIG) prior which integrates out the residual variance $\sigma^2$ analytically, producing t-distributed marginals instead of Gaussian. This tests whether the NIG prior interacts properly with the stochastic LD variance inflation correction.

## Background: Stochastic LD Sketch

Given a standardized genotype matrix $X_{\text{std}} \in \mathbb{R}^{n \times p}$, the true LD matrix is:

$$R = \frac{1}{n-1} X_{\text{std}}^\top X_{\text{std}}$$

The stochastic sketch approximates $R$ using a random projection matrix $W \in \mathbb{R}^{n \times B}$ with $W_{ij} \sim N(0, 1/n)$:

$$U = X_{\text{std}}^\top W \in \mathbb{R}^{p \times B}$$

Each column of $U$ is approximately drawn from $N(0, R)$, so $\hat{R} = U U^\top / B \to R$ as $B \to \infty$.

In `susie_rss()`, we pass `X = t(U)` (a $B \times p$ matrix) and the function treats it as a "genotype-like" input whose cross-product approximates the LD structure.

## SER Variance Inflation Correction

Under the null hypothesis $H_0: \beta_j = 0$, the z-score approximately satisfies:

$$\hat{z}_j = \sum_k R_{jk} z_k$$

When $R$ is replaced by the noisy sketch $\hat{R}$, the variance of $\hat{z}_j$ under $H_0$ is inflated:

$$\text{Var}(\hat{z}_j \mid H_0) = \left(1 + \frac{1}{B}\right) \sigma^2_{j,0} + \frac{R_{jj}}{B} \|R\|_F^2 := \tau_j^2$$

The correction replaces the null variance $\sigma^2_{j,0}$ with $\tau_j^2$ in the single-effect regression Bayes factor. This attenuates the BF for variants whose apparent signal is due to LD noise rather than true association.

Setting `stochastic_ld_sample = B` in `susie_rss()` activates this correction.

In [ ]:
# ============================================================
# Setup: parameters and logging
# ============================================================
library(susieR)

if (requireNamespace("Rfast", quietly = TRUE)) {
  library(Rfast)
  fast_cor <- Rfast::cora
  cat("Using Rfast::cora for correlation matrices (much faster)\n")
} else {
  fast_cor <- cor
  cat("Rfast not available, using base cor() (consider installing Rfast)\n")
}

# --- Simulation parameters ---
p         <- 5000
n         <- 100000
n_causal  <- 4
pve_per   <- 0.0005      # PVE per causal
B_values  <- c(2000, 5000, 10000)
n_rep     <- 50         # set to 250 for full run
L         <- 10
seed      <- 999

# --- Log file for monitoring progress from the terminal ---
# When running via jupyter nbconvert, cat() output is captured into the
# notebook and NOT shown on screen. To monitor progress, open a second
# terminal and run:   tail -f stochastic_ld_benchmark.log
LOG_FILE <- "stochastic_ld_benchmark.log"
log_con  <- file(LOG_FILE, open = "w")

log_msg <- function(fmt, ...) {
  msg <- sprintf(paste0("[%s] ", fmt, "\n"), format(Sys.time(), "%H:%M:%S"), ...)
  cat(msg)                           # notebook cell output
  cat(msg, file = log_con)           # log file for tail -f
  flush(log_con)
}

set.seed(seed)
log_msg("Stochastic LD Benchmark started")
log_msg("  susieR version: %s", packageVersion("susieR"))
log_msg("  Parameters: p=%d, n=%d, n_causal=%d, pve_per=%.3f, n_rep=%d",
        p, n, n_causal, pve_per, n_rep)
log_msg("  B values: %s", paste(B_values, collapse = ", "))
log_msg("  Log file: %s  (run 'tail -f %s' to monitor)", LOG_FILE, LOG_FILE)

## Function Definitions

All simulation functions are defined here. The main execution cell below calls them.

In [ ]:
# ============================================================
# Function: build_ld_structure
# Build block-diagonal AR(1) LD structure
# Returns: list with blocks, block_membership, n_blocks
# ============================================================
build_ld_structure <- function(p, seed = 42) {
  set.seed(seed)
  blocks <- list()
  block_membership <- integer(p)
  pos <- 1
  block_id <- 0
  while (pos <= p) {
    block_id <- block_id + 1
    bsize <- min(sample(20:50, 1), p - pos + 1)
    rho <- runif(1, 0.4, 0.98)
    idx <- pos:(pos + bsize - 1)
    block_membership[idx] <- block_id
    R_block <- rho^abs(outer(1:bsize, 1:bsize, "-"))
    blocks[[block_id]] <- list(idx = idx, R = R_block, rho = rho, size = bsize)
    pos <- pos + bsize
  }
  list(blocks = blocks, block_membership = block_membership, n_blocks = block_id)
}

# ============================================================
# Function: generate_genotypes
# Generate X (n x p) with block-diagonal LD, standardize columns,
# compute true in-sample R = cor(X).
# Returns: list(X, R_true)
# ============================================================
generate_genotypes <- function(n, ld_struct, seed = 43) {
  set.seed(seed)
  p <- sum(sapply(ld_struct$blocks, function(b) b$size))
  log_msg("  Allocating X (%d x %d), ~%.1f GB", n, p, n * p * 8 / 1e9)
  X <- matrix(0, nrow = n, ncol = p)
  n_blocks <- length(ld_struct$blocks)
  for (b in seq_along(ld_struct$blocks)) {
    block <- ld_struct$blocks[[b]]
    L_chol <- chol(block$R)
    Z <- matrix(rnorm(n * block$size), nrow = n, ncol = block$size)
    X[, block$idx] <- Z %*% L_chol
    if (b %% 25 == 0 || b == n_blocks)
      log_msg("    Block %d/%d done (variants %d-%d)",
              b, n_blocks, min(block$idx), max(block$idx))
  }
  log_msg("  Standardizing columns...")
  X <- scale(X)
  attr(X, "scaled:center") <- NULL
  attr(X, "scaled:scale") <- NULL
  log_msg("  Computing R = fast_cor(X) [%d x %d]...", p, p)
  R_true <- fast_cor(X)
  list(X = X, R_true = R_true)
}

# ============================================================
# Function: compute_ld_approximations
# For each B, compute stochastic sketch and subsample R.
# Returns: list(sketches, R_subs)
# ============================================================
compute_ld_approximations <- function(X, B_values, seed = 44) {
  set.seed(seed)
  n <- nrow(X); p <- ncol(X)
  sketches <- list()
  R_subs   <- list()
  for (B in B_values) {
    Bstr <- as.character(B)
    log_msg("  B=%d: generating sketch W(%d x %d), U = X'W (%d x %d)...",
            B, n, B, p, B)
    t0 <- proc.time()
    W <- matrix(rnorm(n * B, sd = 1 / sqrt(n)), nrow = n, ncol = B)
    U <- crossprod(X, W)
    sketches[[Bstr]] <- t(U)
    rm(W, U)
    log_msg("    Sketch done in %.1f sec (%d x %d)",
            (proc.time() - t0)[3], nrow(sketches[[Bstr]]), ncol(sketches[[Bstr]]))
    log_msg("  B=%d: computing subsample R from %d individuals...", B, B)
    t0 <- proc.time()
    R_subs[[Bstr]] <- fast_cor(X[sample(n, B), ])
    log_msg("    Subsample R done in %.1f sec", (proc.time() - t0)[3])
  }
  list(sketches = sketches, R_subs = R_subs)
}

# ============================================================
# Function: simulate_phenotype
# Pick causal variants from different blocks, generate y.
# ============================================================
simulate_phenotype <- function(X, ld_struct, n_causal, pve_per) {
  n <- nrow(X); p <- ncol(X)
  causal_blocks <- sample(ld_struct$n_blocks, n_causal)
  causal_idx <- sapply(causal_blocks, function(b) sample(ld_struct$blocks[[b]]$idx, 1))
  total_pve <- pve_per * n_causal
  sigma_e2 <- 1 - total_pve
  beta_j <- sqrt(pve_per * sigma_e2 / (1 - total_pve))
  beta <- rep(0, p)
  beta[causal_idx] <- beta_j
  y <- as.vector(X %*% beta + rnorm(n, sd = sqrt(sigma_e2)))
  list(y = y, causal_idx = causal_idx, beta = beta)
}

# ============================================================
# Function: compute_zscores
# Marginal z-scores via individual-level univariate regressions.
# ============================================================
compute_zscores <- function(X, y) {
  n <- nrow(X); p <- ncol(X)
  z <- numeric(p)
  for (j in seq_len(p)) {
    fit <- .lm.fit(cbind(1, X[, j]), y)
    bhat <- fit$coefficients[2]
    rss  <- sum(fit$residuals^2)
    se   <- sqrt(rss / (n - 2)) / sqrt(sum((X[, j] - mean(X[, j]))^2))
    z[j] <- bhat / se
  }
  z
}

# ============================================================
# Function: evaluate_fit
# Extract CS, return RAW COUNTS for proper aggregation.
# FDR and power are computed from accumulated counts across replicates.
# ============================================================
evaluate_fit <- function(fit, true_causal) {
  cs <- susie_get_cs(fit)$cs
  n_cs <- length(cs)
  if (n_cs == 0) {
    return(list(n_cs = 0, n_true_cs = 0, n_false_cs = 0,
                n_found = 0, n_causal = length(true_causal), mean_size = 0))
  }
  n_true_cs <- sum(sapply(cs, function(s) any(true_causal %in% s)))
  n_false_cs <- n_cs - n_true_cs
  n_found <- sum(sapply(true_causal, function(c)
                   any(sapply(cs, function(s) c %in% s))))
  list(n_cs = n_cs, n_true_cs = n_true_cs, n_false_cs = n_false_cs,
       n_found = n_found, n_causal = length(true_causal),
       mean_size = mean(sapply(cs, length)))
}

# ============================================================
# Function: run_one_method
# Run susie_rss with given inputs, evaluate, return one-row df.
# ============================================================
run_one_method <- function(method_name, z, causal_idx, n, L, ...) {
  t0 <- proc.time()
  fit <- tryCatch(
    susie_rss(z = z, n = n, L = L, max_iter = 200, verbose = FALSE, ...),
    error = function(e) { log_msg("    %s ERROR: %s", method_name, e$message); NULL }
  )
  if (is.null(fit)) return(NULL)
  ev <- evaluate_fit(fit, causal_idx)
  elapsed <- (proc.time() - t0)[3]
  # Compute per-replicate rates for the log line
  rep_power <- ev$n_found / ev$n_causal
  rep_fdr   <- if (ev$n_cs > 0) ev$n_false_cs / ev$n_cs else 0
  log_msg("    %-25s %5.1fs | power=%.2f  fdr=%.2f  n_cs=%d  size=%.0f",
          method_name, elapsed, rep_power, rep_fdr, ev$n_cs, ev$mean_size)
  data.frame(method = method_name,
             n_cs = ev$n_cs, n_true_cs = ev$n_true_cs, n_false_cs = ev$n_false_cs,
             n_found = ev$n_found, n_causal = ev$n_causal,
             mean_size = ev$mean_size,
             stringsAsFactors = FALSE)
}

# ============================================================
# Function: summarize_results
# Aggregate raw counts across replicates, compute FDR and power.
# ============================================================
summarize_results <- function(results) {
  methods <- unique(results$method)
  # Order methods logically
  method_order <- c("insample_R")
  for (B in B_values) {
    method_order <- c(method_order,
                      paste0("subsample_B", B),
                      paste0("stoch_B", B, "_nocorr"),
                      paste0("stoch_B", B, "_corr"),
                      paste0("stoch_B", B, "_NIG_nocorr"),
                      paste0("stoch_B", B, "_NIG_corr"))
  }
  methods <- method_order[method_order %in% methods]

  summary_rows <- list()
  for (m in methods) {
    rows <- results[results$method == m, ]
    total_cs       <- sum(rows$n_cs)
    total_false_cs <- sum(rows$n_false_cs)
    total_found    <- sum(rows$n_found)
    total_causal   <- sum(rows$n_causal)
    # Weighted mean CS size (weight by n_cs per replicate)
    if (total_cs > 0) {
      avg_size <- sum(rows$mean_size * rows$n_cs) / total_cs
    } else {
      avg_size <- 0
    }
    summary_rows[[length(summary_rows) + 1]] <- data.frame(
      method = m,
      total_cs = total_cs,
      total_false_cs = total_false_cs,
      total_found = total_found,
      total_causal = total_causal,
      FDR = if (total_cs > 0) total_false_cs / total_cs else 0,
      power = total_found / total_causal,
      mean_size = avg_size,
      n_rep = nrow(rows),
      stringsAsFactors = FALSE
    )
  }
  do.call(rbind, summary_rows)
}

# ============================================================
# Function: run_simulation
# Main loop: for each replicate, simulate phenotype, compute z,
# run all methods, collect results. Print final summary.
# ============================================================
run_simulation <- function(X, R_true, sketches, R_subs, ld_struct,
                           B_values, n_rep, n_causal, pve_per, L, seed) {
  set.seed(seed)
  n <- nrow(X)
  results <- list()
  rep_times <- numeric()
  t_total <- proc.time()

  for (rep_i in seq_len(n_rep)) {
    eta_str <- ""
    if (rep_i > 1) {
      eta_min <- mean(rep_times) * (n_rep - rep_i + 1) / 60
      eta_str <- sprintf("  (ETA: %.0f min)", eta_min)
    }
    log_msg("=== Replicate %d / %d ===%s", rep_i, n_rep, eta_str)
    t_rep <- proc.time()

    # Simulate phenotype
    sim <- simulate_phenotype(X, ld_struct, n_causal, pve_per)
    log_msg("  Causal variants: %s", paste(sim$causal_idx, collapse = ", "))

    # Compute z-scores
    t_z <- proc.time()
    log_msg("  Computing z-scores (%d variants)...", ncol(X))
    z <- compute_zscores(X, sim$y)
    log_msg("  z-scores done in %.1f sec, max|z|=%.2f",
            (proc.time() - t_z)[3], max(abs(z)))

    # Gold standard
    row <- run_one_method("insample_R", z, sim$causal_idx, n, L, R = R_true)
    if (!is.null(row)) { row$rep <- rep_i; results[[length(results) + 1]] <- row }

    # For each B
    for (B in B_values) {
      Bstr <- as.character(B)
      # Subsample
      row <- run_one_method(sprintf("subsample_B%d", B), z, sim$causal_idx,
                            n, L, R = R_subs[[Bstr]])
      if (!is.null(row)) { row$rep <- rep_i; results[[length(results) + 1]] <- row }
      # Stochastic, no correction
      row <- run_one_method(sprintf("stoch_B%d_nocorr", B), z, sim$causal_idx,
                            n, L, X = sketches[[Bstr]])
      if (!is.null(row)) { row$rep <- rep_i; results[[length(results) + 1]] <- row }
      # Stochastic, with correction
      row <- run_one_method(sprintf("stoch_B%d_corr", B), z, sim$causal_idx,
                            n, L, X = sketches[[Bstr]], stochastic_ld_sample = B)
      if (!is.null(row)) { row$rep <- rep_i; results[[length(results) + 1]] <- row }
      # Stochastic + NIG prior, no correction
      row <- run_one_method(sprintf("stoch_B%d_NIG_nocorr", B), z, sim$causal_idx,
                            n, L, X = sketches[[Bstr]],
                            estimate_residual_method = "Servin_Stephens")
      if (!is.null(row)) { row$rep <- rep_i; results[[length(results) + 1]] <- row }
      # Stochastic + NIG prior, with correction
      row <- run_one_method(sprintf("stoch_B%d_NIG_corr", B), z, sim$causal_idx,
                            n, L, X = sketches[[Bstr]], stochastic_ld_sample = B,
                            estimate_residual_method = "Servin_Stephens")
      if (!is.null(row)) { row$rep <- rep_i; results[[length(results) + 1]] <- row }
    }

    elapsed_rep <- (proc.time() - t_rep)[3]
    rep_times <- c(rep_times, elapsed_rep)
    log_msg("  Replicate %d done in %.1f sec (%.1f min)",
            rep_i, elapsed_rep, elapsed_rep / 60)
  }

  elapsed_total <- (proc.time() - t_total)[3]
  log_msg("All %d replicates completed in %.1f min (avg %.1f sec/rep)",
          n_rep, elapsed_total / 60, mean(rep_times))

  results_df <- do.call(rbind, results)

  # --- Final summary (accumulated across all replicates) ---
  summary_df <- summarize_results(results_df)
  log_msg("")
  log_msg("====================================================")
  log_msg("  FINAL SUMMARY (accumulated over %d replicates)", n_rep)
  log_msg("====================================================")
  log_msg("%-30s %6s %6s %6s %8s %8s",
          "method", "FDR", "power", "n_cs", "false", "size")
  log_msg("%-30s %6s %6s %6s %8s %8s",
          "------------------------------", "------", "------", "------", "--------", "--------")
  for (i in seq_len(nrow(summary_df))) {
    s <- summary_df[i, ]
    log_msg("%-30s %6.3f %6.3f %6d %8d %8.0f",
            s$method, s$FDR, s$power, s$total_cs, s$total_false_cs, s$mean_size)
  }
  log_msg("====================================================")

  list(results = results_df, summary = summary_df)
}

log_msg("All functions defined.")

## Run Simulation

Execution proceeds in four phases:
1. Build LD structure (fast)
2. Generate genotype matrix and true LD (slow — several minutes)
3. Pre-compute stochastic sketches and subsample LD for each $B$
4. Run replicates: simulate phenotype, compute z-scores, run all methods

**To monitor progress**, open a second terminal and run:
```bash
tail -f stochastic_ld_benchmark.log
```

In [ ]:
# ============================================================
# Phase 1: Build LD structure
# ============================================================
log_msg("Phase 1: Building LD structure (p=%d)...", p)
ld_struct <- build_ld_structure(p, seed = seed)
log_msg("  %d blocks, sizes %d-%d, rho %.2f-%.2f",
        ld_struct$n_blocks,
        min(sapply(ld_struct$blocks, function(b) b$size)),
        max(sapply(ld_struct$blocks, function(b) b$size)),
        min(sapply(ld_struct$blocks, function(b) b$rho)),
        max(sapply(ld_struct$blocks, function(b) b$rho)))

# ============================================================
# Phase 2: Generate genotypes and true LD
# ============================================================
log_msg("Phase 2: Generating genotypes (%d x %d)...", n, p)
t0 <- proc.time()
geno <- generate_genotypes(n, ld_struct, seed = seed + 1)
X      <- geno$X
R_true <- geno$R_true
rm(geno)
log_msg("Phase 2 complete in %.1f min", (proc.time() - t0)[3] / 60)

# ============================================================
# Phase 3: Pre-compute LD approximations
# ============================================================
log_msg("Phase 3: Computing LD approximations for B = {%s}...",
        paste(B_values, collapse = ", "))
t0 <- proc.time()
ld_approx <- compute_ld_approximations(X, B_values, seed = seed + 2)
sketches <- ld_approx$sketches
R_subs   <- ld_approx$R_subs
rm(ld_approx)
log_msg("Phase 3 complete in %.1f min", (proc.time() - t0)[3] / 60)

# ============================================================
# Phase 4: Run simulation replicates
# ============================================================
n_methods <- 1 + length(B_values) * 5  # insample + 5 per B (subsample, stoch x2, NIG x2)
log_msg("Phase 4: Running %d replicates x %d methods...", n_rep, n_methods)
sim_output <- run_simulation(X, R_true, sketches, R_subs, ld_struct,
                             B_values, n_rep, n_causal, pve_per, L,
                             seed = seed + 100)
results <- sim_output$results
summary_df <- sim_output$summary
log_msg("Simulation complete. %d result rows.", nrow(results))

In [ ]:
# ============================================================
# Save results to RDS (both per-replicate and summary)
# ============================================================
rds_file <- sprintf("stochastic_ld_benchmark_n%d_p%d_nrep%d.rds", n, p, n_rep)
saveRDS(list(results = results, summary = summary_df), file = rds_file)
log_msg("Results saved to %s", rds_file)
close(log_con)  # close log file

cat("\n=== FINAL SUMMARY ===\n")
print(summary_df[, c("method", "FDR", "power", "total_cs", "total_false_cs", "mean_size")],
      row.names = FALSE)

## Results and Visualization

Three plots:
1. **FDR bar plot** — with a horizontal line at the nominal 0.05 level
2. **Power bar plot**
3. **Power vs. FDR scatter** — summarizing the trade-off

In [ ]:
# ============================================================
# Load and display summary (from accumulated raw counts)
# ============================================================
# To reload from saved file:
# dat <- readRDS(rds_file)
# results <- dat$results
# summary_df <- dat$summary

# Order methods logically
method_order <- c("insample_R")
for (B in B_values) {
  method_order <- c(method_order,
                    paste0("subsample_B", B),
                    paste0("stoch_B", B, "_nocorr"),
                    paste0("stoch_B", B, "_corr"),
                    paste0("stoch_B", B, "_NIG_nocorr"),
                    paste0("stoch_B", B, "_NIG_corr"))
}
summary_df$method <- factor(summary_df$method, levels = method_order)
summary_df <- summary_df[order(summary_df$method), ]

cat("\n=== Summary (accumulated over all replicates) ===\n")
print(summary_df[, c("method", "FDR", "power", "total_cs", "total_false_cs",
                      "total_found", "total_causal", "mean_size")], row.names = FALSE)

In [ ]:
# ============================================================
# FDR bar plot (from accumulated counts)
# ============================================================
method_colors <- c("insample_R" = "gray50")
for (B in B_values) {
  method_colors[paste0("subsample_B", B)]            <- "steelblue"
  method_colors[paste0("stoch_B", B, "_nocorr")]     <- "tomato"
  method_colors[paste0("stoch_B", B, "_corr")]       <- "seagreen"
  method_colors[paste0("stoch_B", B, "_NIG_nocorr")] <- "darkorange"
  method_colors[paste0("stoch_B", B, "_NIG_corr")]   <- "mediumpurple"
}

par(mar = c(12, 4, 3, 1))
bp <- barplot(summary_df$FDR, names.arg = summary_df$method,
              col = method_colors[as.character(summary_df$method)],
              las = 2, ylab = "FDR", main = "False Discovery Rate by Method",
              ylim = c(0, max(summary_df$FDR, 0.1) * 1.2))
abline(h = 0.05, col = "darkred", lty = 2, lwd = 2)
text(max(bp), 0.05, "  0.05", adj = c(0, -0.3), col = "darkred", cex = 0.8)
legend("topright",
       legend = c("In-sample R", "Subsample", "Stoch (no corr)", "Stoch (corrected)",
                  "NIG (no corr)", "NIG (corrected)"),
       fill = c("gray50", "steelblue", "tomato", "seagreen",
                "darkorange", "mediumpurple"), cex = 0.7, bty = "n")

In [ ]:
# ============================================================
# Power bar plot (from accumulated counts)
# ============================================================
par(mar = c(12, 4, 3, 1))
bp <- barplot(summary_df$power, names.arg = summary_df$method,
              col = method_colors[as.character(summary_df$method)],
              las = 2, ylab = "Power", main = "Power by Method",
              ylim = c(0, 1))
legend("topright",
       legend = c("In-sample R", "Subsample", "Stoch (no corr)", "Stoch (corrected)",
                  "NIG (no corr)", "NIG (corrected)"),
       fill = c("gray50", "steelblue", "tomato", "seagreen",
                "darkorange", "mediumpurple"), cex = 0.7, bty = "n")

In [ ]:
# ============================================================
# Power vs FDR scatter plot (from accumulated counts)
# ============================================================
par(mar = c(5, 4, 3, 1))
method_pch <- c("insample_R" = 16)
pch_vals <- c(17, 15, 18)
for (i in seq_along(B_values)) {
  B <- B_values[i]
  method_pch[paste0("subsample_B", B)]            <- pch_vals[i]
  method_pch[paste0("stoch_B", B, "_nocorr")]     <- pch_vals[i]
  method_pch[paste0("stoch_B", B, "_corr")]       <- pch_vals[i]
  method_pch[paste0("stoch_B", B, "_NIG_nocorr")] <- pch_vals[i]
  method_pch[paste0("stoch_B", B, "_NIG_corr")]   <- pch_vals[i]
}

plot(summary_df$FDR, summary_df$power, type = "n",
     xlim = c(0, max(summary_df$FDR, 0.1) * 1.3), ylim = c(0, 1),
     xlab = "FDR", ylab = "Power", main = "Power vs FDR")
points(summary_df$FDR, summary_df$power,
       col = method_colors[as.character(summary_df$method)],
       pch = method_pch[as.character(summary_df$method)], cex = 2)
text(summary_df$FDR, summary_df$power, labels = summary_df$method,
     pos = 4, cex = 0.5, col = "gray30")
abline(v = 0.05, col = "darkred", lty = 2, lwd = 2)
text(0.05, 0, "FDR = 0.05", adj = c(-0.1, -0.3), col = "darkred", cex = 0.8)
legend("bottomright",
       legend = c("In-sample R", "Subsample", "Stoch (no corr)", "Stoch (corrected)",
                  "NIG (no corr)", "NIG (corrected)",
                  paste0("B=", B_values)),
       col = c("gray50", "steelblue", "tomato", "seagreen",
               "darkorange", "mediumpurple", rep("black", 3)),
       pch = c(16, 16, 16, 16, 16, 16, pch_vals),
       pt.cex = 1.5, cex = 0.6, bty = "n")

## Command-Line Execution

```bash
cd /path/to/notebooks

# Terminal 1: run the notebook
jupyter nbconvert --execute --to notebook \
    --output stochastic_ld_benchmark_executed.ipynb \
    --ExecutePreprocessor.timeout=7200 \
    stochastic_ld_benchmark.ipynb

# Terminal 2: monitor progress
tail -f stochastic_ld_benchmark.log
```

For the full run, edit the setup cell to set `n_rep <- 100`.

Results are saved to `stochastic_ld_benchmark_n100000_p5000_nrep100.rds`.